#### Directories etc

In [21]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import random
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
from collections import defaultdict
import random
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import re
import os
import subprocess
import copy
from Comparative_Analysis import ORF_Functions as orffn
from random import sample
pd.options.mode.chained_assignment = None  # default='warn'
import ete3
import matplotlib.pyplot as plt
from statistics import mode, median
from scipy.stats import chi2
import shutil

In [2]:
project_dir = 'F:/Project_Data/Project_11'
seq_dir = 'F:/Datasets/Actinobacteria_Ref_Rep_Lev_Complete'
dictionary_dir = 'F:/Datasets/CRYPTIC_DATA/Cryptic_Dictionaries'
tb_species = 'NC_000962.3' 
tb_genome_filename = 'GCF_000195955.2_ASM19595v2_genomic.gbff'
mycobrowser_dir = 'F:/Datasets/Data_From_Publications'
num_cores = 16
core_numbers = list(range(1, num_cores+1))

In [9]:
for record in SeqIO.parse(seq_dir + '/' + tb_genome_filename, "genbank"):
    full_sequence = str(record.seq)

In [4]:
with open(project_dir + '/Thoth_Full_Run/zero_and_non_zero_mutation_counts.pkl', 'rb') as f:
    full_sample_zero_and_non_zero_mutation_counts = pickle.load(f)   

In [46]:
compass_df = pd.read_csv('F:/Datasets/CRYPTIC_DATA/COMPASS.csv', names = ['species', 'start', 'stop','count'])
compass_locations = []

for i, r in compass_df.iterrows():
    for k in range(r['start'], r['stop']):     #  Should te start-1 but seems to ve inconsistency with mutations!!
        compass_locations.append(k)

compass_sample = random.sample(compass_locations, 60)
        

In [63]:
temp = []
for n, x in enumerate(full_sample_zero_and_non_zero_mutation_counts):
    if x > 2000:
        temp.append(['pos_'+str(n), full_sequence[n-20:n+20]])
util.produce_fasta_file(temp, project_dir +'/highly_mutated_positions.faa')
util.produce_fasta_file([['H37Rv', full_sequence]], project_dir +'/highly_mutated_positions_full_sequence.faa')

temp = []
for x in (compass_sample):
    temp.append(['compass_pos_'+str(x), full_sequence[x-20:x+20]])
util.produce_fasta_file(temp, project_dir +'/compass_sample.faa')

100%|██████████| 60/60 [00:00<?, ?it/s]


In [64]:
w_d = os.getcwd()
os.chdir("F:/")
subprocess.run('cd '+ project_dir + ' &  makeblastdb -in ' + project_dir + '/highly_mutated_positions_full_sequence.faa -dbtype nucl -out hmpos_fs', shell=True, capture_output = True)
os.chdir(w_d)
if not(os.path.exists('F:/Datasets/BLAST/hmpos_fs')):
    os.makedirs('F:/Datasets/BLAST/hmpos_fs')
files = util.list_files(project_dir)
for file in files:
    if 'hmpos' in file:
        shutil.move(project_dir+'/'+file, 'F:/Datasets/BLAST/hmpos_fs/' + file)  
for file in ['highly_mutated_positions.faa','highly_mutated_positions_full_sequence.faa','compass_sample.faa']:
    shutil.move(project_dir+'/'+file, 'F:/Datasets/BLAST/hmpos_fs/' + file)  


In [69]:
#-evalue 1000
w_d = os.getcwd()
os.chdir("F:/")
subprocess.run('cd f:\\Datasets\\BLAST\\hmpos_fs & blastn -task blastn-short -query highly_mutated_positions.faa -db hmpos_fs -evalue 1e-5 -out hits.csv  -outfmt "10 qaccver saccver qlen slen pident length mismatch gapopen qstart qend sstart send evalue bitscore" -num_threads 16', shell=True, capture_output = True)
subprocess.run('cd f:\\Datasets\\BLAST\\hmpos_fs & blastn -task blastn-short -query compass_sample.faa -db hmpos_fs -evalue 1e-5 -out compass_hits.csv -outfmt "10 qaccver saccver qlen slen pident length mismatch gapopen qstart qend sstart send evalue bitscore" -num_threads 16', shell=True, capture_output = True)
os.chdir(w_d)
blast_results = pd.read_csv('F:/Datasets/BLAST/hmpos_fs/hits.csv', header = None)
blast_results.columns = ['query_ref', 'target_ref', 'query_length', 'subject_length', 'percent_identical_matches','alignment_length', 'number_mismatches', 'number_of_gap_openings', 
                         'query_start_alignment', 'query_end_alignment', 'target_start_alignment', 'target_end_alignment', 'e_value', 'bit_score']
blast_results.to_csv('F:/Datasets/BLAST/hmpos_fs/hits.csv')
blast_results = pd.read_csv('F:/Datasets/BLAST/hmpos_fs/compass_hits.csv', header = None)
blast_results.columns = ['query_ref', 'target_ref', 'query_length', 'subject_length', 'percent_identical_matches','alignment_length', 'number_mismatches', 'number_of_gap_openings', 
                         'query_start_alignment', 'query_end_alignment', 'target_start_alignment', 'target_end_alignment', 'e_value', 'bit_score']
blast_results.to_csv('F:/Datasets/BLAST/hmpos_fs/compass_hits.csv')



In [32]:
blast_results = pd.read_csv('F:/Datasets/BLAST/Self_BLAST/' + species + '/hits.csv', header = None)
blast_results.columns = ['query_ref', 'target_ref', 'query_length', 'subject_length', 'percent_identical_matches','alignment_length', 'number_mismatches', 'number_of_gap_openings', 
                         'query_start_alignment', 'query_end_alignment', 'target_start_alignment', 'target_end_alignment', 'e_value', 'bit_score']

blast_results['hit_count'] = blast_results.groupby('query_ref')['target_ref'].transform('count')
blast_results = blast_results[blast_results['hit_count'] > 1]

repeat_regions = []
blast_results['annot_features']=''
for i, r in blast_results.iterrows():
    start1 = min(r['target_start_alignment'],r['target_end_alignment'])
    end1 = max(r['target_start_alignment'],r['target_end_alignment'])
    feature_matches = []
    for (locus, ortholog_locus, product, feature, start, stop) in annotated_regions_dict[r['target_ref']]:
        if start< end1 and stop > start1:
            overlap = (min(end1, stop) - max(start1, start))/ (end1-start1)
            #Don't want to output ortholog info as it clutters!
            #feature_matches.append([locus, ortholog_locus, product, feature, overlap])
            feature_matches.append([locus, product, feature, overlap])
            if ('repeat' in feature) or ('mobile' in feature):
                repeat_regions.append(r['query_ref'])
    blast_results.at[i,'annot_features'] = feature_matches